# Ealuation

This notebook includes evaluation of the Whisper Large-v2 model and a fine-tuned version of the same model. The audio used is a simulated emergency call based on an anonymized real scenario. It is done by loading the fine-tuned verison, then the original and have them both generate a transcript. The Word Error Rate (WER) is then calculated and compared between both models.

The fine-tuned model is only trained on approximately 7 minutes of audio, similar to the emergency call used for evaluation. It is not expected that the results show any significant improvements.

In [1]:
with open("lydlogg2_transcript.txt") as f:
    transcription = f.read()[:-1]

In [2]:
transcription

'113 ambulansen, er pasienten våken? Ja. Ja, ja. Eller ja, jeg tror det. Det er Arnulf Thoraldsdottir som ringer og jeg ringer om kona mi, Margit, hun er 87 år og ligger på bakken og kommer seg ikke opp hun nå. Jeg vet ikke helt hva jeg skal gjøre, jeg. Nei, men jeg skal hjelpe deg. Hvilken adresse er det dere er på? Vi er i øvre frydendal 437. Ute på frydendal ute i Asker der. Ja er dere ute? Nei, vi er inne. Inne i leiligheten vår. I 6. etasje i leiligheten vår. Det står Thoraldsdottir på ringeklokka nede. Ja, men hva er det som har skjedd for noe da? Nei jeg hadde bare hvert på toalettet og når jeg kom ut så ligger hun på bakken her. Margit, er du våken? Margit? Ja da, hun ser på meg altså. Ja. Men hun ligger på bakken. Jeg får henne ikke opp jeg altså. Jeg er pjuskete selv, så jeg klarer ikke løfte opp Margit. Hun er stor dame. Svarer hun ordentlig når du snakker til henne? Margit? Ja, Margit? Ja da, hun svarer. Hun sier hun har vondt da. Hun har slått seg litt. Hun har vondt i hof

In [3]:
def generate_results(pipe):
    

    results = []        

    result = pipe("Lydlogg 2 test 113.wav", return_timestamps=True)["chunks"]
    
    results.append(result)
    

    print("DONE GENERATING RESULTS")
    return results

In [4]:
%%time
import torch
from transformers import WhisperForConditionalGeneration, WhisperProcessor, pipeline
import gc

device = 0 if torch.cuda.is_available() else "cpu"

pipe = pipeline(
    "automatic-speech-recognition",
    model="amk-s2t/amk-whisper-local",
    chunk_length_s=30,
    device=device,
    tokenizer="amk-s2t/amk-whisper-local",
    generate_kwargs={
        "language":"<|no|>",
        "task":"transcribe"
    }
)

CPU times: user 24.5 s, sys: 11.3 s, total: 35.8 s
Wall time: 34.9 s


In [5]:
%%time
results_ft = generate_results(pipe)

/home/jens/anaconda3/envs/amk/lib/python3.8/site-packages/transformers/generation/utils.py:1344: UserWarning: Using `max_length`'s default (448) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


DONE GENERATING RESULTS
CPU times: user 1min 42s, sys: 7.56 s, total: 1min 50s
Wall time: 1min 50s


Clearing the VRAM in order to load the original model.

In [6]:
del pipe
gc.collect()
torch.cuda.empty_cache

<function torch.cuda.memory.empty_cache() -> None>

In [7]:
pipe_org = pipeline(
    "automatic-speech-recognition",
    model="openai/whisper-large-v2",
    chunk_length_s=30,
    device=device,
    generate_kwargs={
        "language":"<|no|>",
        "task":"transcribe"
    }
)

In [8]:
results_org = generate_results(pipe_org)

/home/jens/anaconda3/envs/amk/lib/python3.8/site-packages/transformers/generation/utils.py:1344: UserWarning: Using `max_length`'s default (448) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


DONE GENERATING RESULTS


## Clean whisper output

Whisper outputs the results as many segments with timestamps. Therefore we must clean the output so that it is rather formatted as a single text string.

In [12]:
def clean_results(batch):
    new_batch = []
    for part in batch:
        clean_part = ""
        for segment in part:
            clean_part += segment["text"]
        
        new_batch.append(clean_part)
    
    return new_batch

results_ft = clean_results(results_ft)
results_org = clean_results(results_org)

## Displaying the evaluation results

In [13]:
from IPython.display import display, HTML
import pandas as pd


df = pd.DataFrame()

#inital = {0:dataset.remove_columns(["audio"])[0]["transcription"] + dataset.remove_columns(["audio"])[1]["transcription"]}
df["transcription"] = {0:transcription}
df["finetuned_predictions"] = results_ft[0]
df["original_predictions"] = results_org[0]
display(HTML(df.to_html()))

In [14]:
processor = WhisperProcessor.from_pretrained("amk-s2t/amk-whisper-local")

results = {
    "reference": [],
    "ft_prediction": [],
    "org_prediction": []
}

for i, part in enumerate(results_ft):
    clean = ""
    for data in part:
        clean += data
    results["ft_prediction"].append(processor.tokenizer._normalize(clean))
    results["reference"].append(processor.tokenizer._normalize(df["transcription"][i]))
    

In [15]:
gc.collect()
torch.cuda.empty_cache

<function torch.cuda.memory.empty_cache() -> None>

In [16]:
for i, part in enumerate(results_org):
    clean = ""
    for data in part:
        clean += data
    results["org_prediction"].append(processor.tokenizer._normalize(clean))
    

In [17]:
from evaluate import load

wer = load("wer")

wer_ft = 100 * wer.compute(references=results["reference"], predictions=results["ft_prediction"])
wer_org = 100 * wer.compute(references=results["reference"], predictions=results["org_prediction"])
print("Finetuned:", wer_ft, "\nOriginal:", wer_org)

Finetuned: 32.9443447037702 
Original: 33.48294434470377
